This script matches the block with the TAZ and outputs the TAZ table with corresponding information

In [39]:
# Libraries and dependencies
import numpy as np
import pandas as pd

In [40]:
# Read in the input files:
# Parse in the dataframe and skip the first row
new_york_df = pd.read_csv('new_york_size.csv', skiprows=1)
new_york_df.head()
new_york_df['Id2'] = new_york_df['Id2'].astype('int')
new_york_df['avg_hh'] = new_york_df['total hhsize']/new_york_df['total hh num']
new_york_df.head()
# Further clean the data by eliminating rows containing NaN:
new_york_df = new_york_df[~np.isnan(new_york_df).any(axis=1)]
# Calculate the percentage of different numbers of households:
for i in range(1,7):
    new_york_df['percentage_' + str(i)] = new_york_df[str(i)]/new_york_df['total hhsize']
new_york_df.head()

,Id2,1,2,3,4,5,6,7,total hhsize,total hh num,avg_hh,percentage_1,percentage_2,percentage_3,percentage_4,percentage_5,percentage_6
2,-167250853,58,86,48,72,0,20,5,289,1052,0.274715,0.200692,0.297578,0.166090,0.249135,0.000000,0.069204
3,-167250852,224,262,122,13,68,0,0,689,1618,0.425834,0.325109,0.380261,0.177068,0.018868,0.098694,0.000000
5,-167250843,86,63,15,58,0,0,0,222,534,0.415730,0.387387,0.283784,0.067568,0.261261,0.000000,0.000000
6,-167250842,313,349,177,55,22,0,0,916,2245,0.408018,0.341703,0.381004,0.193231,0.060044,0.024017,0.000000
7,-167250841,442,229,59,65,16,0,0,811,1547,0.524240,0.545006,0.282367,0.072750,0.080148,0.019729,0.000000


In [45]:
# Read in the TAZ data:
TAZ = pd.read_csv('TAZ_Household_size-1.csv')
# The first 318 TAZ belong to New York
new_york_TAZ = TAZ.head(318)
new_york_TAZ.head()

,TAZ,HHNUM,HHSIZE
0,1,4347.4,2.036
1,2,2215.2,2.216
2,3,0.0,0.000
3,4,722.0,1.906
4,5,3733.0,1.676


In [42]:
# Start matching the HH_SIZE for the avg_hh in the new_york_TAZ
# Construct a dict to store all the ID of the residential block and its average hh value:
block_dict = {}
for block_row in new_york_df.itertuples(index=True, name='Pandas'):
    block_dict[getattr(block_row, "avg_hh")] =  getattr(block_row, "Id2")
# print(block_dict)  
matches = {}

In [43]:
# Iterate through the TAZ table:
for taz_row in new_york_TAZ.itertuples(index = True, name = 'Pandas'):
    
    current_hh_target = getattr(taz_row, 'HHSIZE')
    
    current_taz_id = getattr(taz_row, 'TAZ')
    
    # Find the closest element in the list according to the target number
    closest_key = min(block_dict.keys(), key=lambda x:abs(x-current_hh_target))
    
    # Retrieve the value stored corresponding to the key, which is the block id:
    current_block_id = block_dict[closest_key]
    matches[current_taz_id] = current_block_id
    
    
    # Remove the items from block_dict since we could not use the same block_id again:
    block_dict.pop(closest_key)    
print(matches)

{1: -167118855, 2: -167133862, 3: -166955863, 4: -167126856, 5: -167164857, 6: -167174861, 7: -167174862, 8: -167106840, 9: -167136861, 10: -167095858, 11: -167220857, 12: -167113862, 13: -167186856, 14: -167174860, 15: -167190863, 16: -167047862, 17: -167156863, 18: -167115862, 19: -167161860, 20: -167128856, 21: -167103860, 22: -167175862, 23: -167069861, 24: -167131861, 25: -167154859, 26: -167106841, 27: -167093860, 28: -167114861, 29: -167175863, 30: -167159859, 31: -167134856, 32: -167161861, 33: -167113859, 34: -167214860, 35: -167164858, 36: -167069859, 37: -167186858, 38: -167214859, 39: -167114856, 40: -167118857, 41: -167205863, 42: -167113860, 43: -167164863, 44: -167173861, 45: -167118863, 46: -167106851, 47: -167222842, 48: -167164860, 49: -167172862, 50: -167125862, 51: -167186861, 52: -167179859, 53: -167126858, 54: -167125859, 55: -167075863, 56: -167154862, 57: -167128863, 58: -167165859, 59: -167146849, 60: -167170862, 61: -167165862, 62: -167171861, 63: -167179863, 

In [48]:
# Fill in the table:
new_york_TAZ.reset_index(drop = True, inplace = True)
for row in new_york_TAZ
    